In [ ]:
# #To be used on cloud platforms

# ! python -c "import pykeen" || pip install pykeen
# ! python -c "import scprep" || pip install scprep

In [16]:
import os, pandas as pd, numpy as np, scanpy, networkx, scprep
import rpy2, rpy2.situation


In [17]:
print(os.getcwd())
data_dir = f"{os.getcwd()}/data"
kge_dir = f"{os.getcwd()}/KGEs"

/Users/ferran/PhD/yaleCollab


# DataBases

The integrated database is the object ingested by pykeen's *TriplesFactory*.

In [3]:
print(pd.read_csv(f"{data_dir}/dCellChat.tsv", sep="\t").head())
print(pd.read_csv(f"{data_dir}/dSTRING.tsv", sep="\t").head())

    head       relation    tail pathway
0  Tgfb1  LRinteraction  Tgfbr1    TGFb
1  Tgfb1  LRinteraction  Tgfbr2    TGFb
2  Tgfb2  LRinteraction  Tgfbr1    TGFb
3  Tgfb2  LRinteraction  Tgfbr2    TGFb
4  Tgfb3  LRinteraction  Tgfbr1    TGFb
    head  relation   tail
0  Gnai3  FIstring  Prkca
1  Gnai3  FIstring  Rgs18
2  Gnai3  FIstring  Gpsm2
3  Gnai3  FIstring  Adrb3
4  Gnai3  FIstring  Gnat2


In [4]:
pd.read_csv(f"{data_dir}/dINT_simpl.tsv", sep="\t", header=None)

,0,1,2
0,Tgfb1,LRinteraction,Tgfbr1
1,Tgfb1,LRinteraction,Tgfbr2
2,Tgfb2,LRinteraction,Tgfbr1
3,Tgfb2,LRinteraction,Tgfbr2
4,Tgfb3,LRinteraction,Tgfbr1
...,...,...,...
439018,Gm28635,FIstring,Arhgap35
439019,Gm28635,FIstring,Cdh17
439020,Gm28635,FIstring,Kif5b
439021,Gm28635,FIstring,Tjp3


From the integrated database, let's make a pandas dataframe that has only the 
source and target columns and then use that to make a NetworkX Graph object 
with `from_pandas_edgelist`

Then, once the scRNAseq has been loaded in, we can also subset the genes in 
this dataframe to those present in the scRNA-seq count matrix.

In [8]:
dEdgeList = pd.read_csv(f"{data_dir}/dINT_simpl.tsv", sep="\t", header=None)[[0,2]]
print(dEdgeList.shape)

#Filter rows if either column not in list(dGenesSEQ)
dEdgeList = dEdgeList.loc[dEdgeList[0].isin(list(dGenesSeq)) & dEdgeList[2].isin(list(dGenesSeq))]
dEdgeList

(439023, 2)


,0,2
0,Tgfb1,Tgfbr1
1,Tgfb1,Tgfbr2
2,Tgfb2,Tgfbr1
3,Tgfb2,Tgfbr2
4,Tgfb3,Tgfbr1
...,...,...
438929,Paqr8,Efhc1
438930,Teddm3,Tmem198b
438931,Teddm3,Slc7a15
438943,Cys1,Fut2


# scRNAseq data

We will have to use rpy to import the seurat object with the data. From there we already have code (from RNA Velo NBs) to extract metadata. Need to extract count matrices.

In [18]:
for i in rpy2.situation.iter_info(): # Print Rpy2 info
    print(i)

rpy2 version:
3.4.5
Python version:
3.9.13 (main, Aug  7 2022, 01:33:23) 
[Clang 13.1.6 (clang-1316.0.21.2.5)]
Looking for R's HOME:
    Environment variable R_HOME: /usr/local/Cellar/r/4.2.1_2/lib/R
    Calling `R RHOME`: /usr/local/Cellar/r/4.2.1_4/lib/R
    Environment variable R_LIBS_USER: None
R's additions to LD_LIBRARY_PATH:

R version:
    In the PATH: R version 4.2.1 (2022-06-23) -- "Funny-Looking Kid"
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:


FileNotFoundError: [Errno 2] No such file or directory: '/usr/local/Cellar/r/4.2.1_2/lib/R/bin/R'

## Prep data

In [3]:
%load_ext rpy2.ipython

In [4]:
%%R -i data_dir
library(tidyverse)
library(Seurat)
library(SeuratDisk)

paste0(data_dir)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

R[write to console]: Attaching SeuratObject

R[write to console]: Attaching sp



[1] "/Users/ferran/PhD/yaleCollab/data"


In [5]:
%%R -o dGenesSeq

d_fibWT = readRDS(paste0(data_dir,"/Omics/","d_fibWT.rds"))
print(d_fibWT)
#Get genes present in Seurat object
dGenesSeq = rownames(d_fibWT@assays$RNA@data)
#Convert seurat object to anndata
SaveH5Seurat(d_fibWT, filename = paste0(data_dir,"/Omics/","d_fibWT.h5Seurat"))
Convert(paste0(data_dir,"/Omics/","d_fibWT.h5Seurat"), dest = "h5ad")



An object of class Seurat 
40940 features across 3698 samples within 2 assays 
Active assay: SCT (20073 features, 10000 variable features)
 1 other assay present: RNA
 3 dimensional reductions calculated: pca, umap, phate


Export list of genes detected in scRNA-seq dataset.
This will be used to subset the Knowledge Graph

In [7]:
len(list(dGenesSEQ))

20867

In [ ]:
dRnaSeq = scanpy.read_h5ad(f"{data_dir}/Omics/d_fibWT.h4ad")

### Normalise and denoise gene expression data

First subset to genes in KG, then use MAGIC to smooth, and then normalise so 
that the gene expression distributions are equally big for each cell.

In [12]:
gKG = dEdgeList.melt()["value"].drop_duplicates()
gKG

0          Tgfb1
2          Tgfb2
4          Tgfb3
12          Bmp2
17          Bmp4
           ...  
348019      Cipc
348023     Paqr8
348024    Teddm3
348026      Cys1
348512     Gpr19
Name: value, Length: 12229, dtype: object

In [13]:
%%R



R[write to console]: Error in library(SeuratDisk) : there is no package called ‘SeuratDisk’

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In fun(libname, pkgname) :
R[write to console]: 
 
R[write to console]:  rgeos: versions of GEOS runtime 3.11.0-CAPI-1.17.0
and GEOS at installation 3.10.3-CAPI-1.16.1differ




Error in library(SeuratDisk) : there is no package called ‘SeuratDisk’


RInterpreterError: Failed to parse and evaluate line '\nlibrary(SeuratDisk)\n'.
R error message: 'Error in library(SeuratDisk) : there is no package called ‘SeuratDisk’'

# Project cell data on gene graph

First we'll need to build and adjancency matrix from the KG. 
Then we will compute DEMD with the adjacency matrix and the distributions of 
normalised gene expression for each cell on our dataset.

## Adjacency matrix from KG

Using [networkx](https://networkx.org/documentation/stable/install.html) package

In [23]:
#First make graph
G = networkx.from_pandas_edgelist(dEdgeList, 0, 2)
G

In [24]:
#Adjacency matrix
AdjMat = networkx.adjacency_matrix(G)
AdjMat

/var/folders/9p/ypn2045x1cxbq9ptcfk4_k7m0000gn/T/ipykernel_32942/3832294983.py:2: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  AdjMat = networkx.adjacency_matrix(G)


<12229x12229 sparse matrix of type '<class 'numpy.int64'>'
	with 347279 stored elements in Compressed Sparse Row format>

## Compute Distances